# Environment Setup

---

### Mount Google Drive folder 

In [84]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Checkout code and model config

In [85]:
!git clone --branch weights https://github.com/hannahg141/ClimateNet.git

Cloning into 'ClimateNet'...
remote: Enumerating objects: 447, done.
remote: Counting objects: 100% (202/202), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 447 (delta 125), reused 149 (delta 85), pack-reused 245
Receiving objects: 100% (447/447), 12.24 MiB | 16.49 MiB/s, done.
Resolving deltas: 100% (246/246), done.


# Train and evaluate model

Check that GPU is connected.

In [86]:
import torch

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

print("\nCuda is available: "+str(torch.cuda.is_available()))

!nvcc --version

Mon Dec  5 01:51:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    43W / 400W |      3MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Move dataset to disk.

In [ ]:
!mkdir /content/Data
!mkdir /content/Data/train
!mkdir /content/Data/test
! ls /content/Data/

In [ ]:
!cp -r --verbose /content/drive/MyDrive/Stanford/CS230/Data/ClimateNet_Engineered/train/ /content/Data/

In [ ]:
!cp -r --verbose /content/drive/MyDrive/Stanford/CS230/Data/ClimateNet_Engineered/test/ /content/Data/

Move shell to ClimateNet directory.

In [88]:
%cd /content/ClimateNet/

/content/ClimateNet


List models:



In [94]:
ls models

TMQ-WS850-VRT850-PLS-.001-jaccard-batch16/  TMQ-WS850-VRT850-PSL-.001-wce/


Check training parameters.

In [95]:
cat models/TMQ-WS850-VRT850-PLS-.001-jaccard-batch16/config.json

{
    "architecture": "cgnet",
    "seed": 42,
    "lr": 0.001,
    "train_batch_size": 16,
    "pred_batch_size": 8,
    "epochs": 20,
    "fields": {
        "TMQ": {
            "mean": 19.2185,
            "std": 15.7346
        },
        "WS850": {
            "mean": 8.70061,
            "std": 5.85448
        },
        "VRT850": {
            "mean": 0.00256632,
            "std": 6.85154
        },
        "PSL": {
            "mean": 100814.414,
            "std": 1461.2227
        }
    },
    "labels": [
      "Background",
      "Tropical Cyclone",
      "Atmospheric River"
    ],
    "loss": "jaccard",
    "save_epochs": true,
    "description": "CGNet model with Jaccard loss and engineered features."
}

In [137]:
!git pull

remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 4 (delta 3), reused 4 (delta 3), pack-reused 0
Unpacking objects: 100% (4/4), done.
From https://github.com/hannahg141/ClimateNet
   5f2f539..e456e2f  weights    -> origin/weights
Updating 5f2f539..e456e2f
Fast-forward
 climatenet/models.py | 2 +-
 1 file changed, 1 insertion(+), 1 deletion(-)


Launch training and evaluation script!

In [138]:
!python3 train_eval.py \
-d /content/Data/ \
-m /content/ClimateNet/models/TMQ-WS850-VRT850-PLS-.001-jl-mb16/


Epoch #1 on device cuda:0:
Loss: 0.82764 (jaccard) :  16% 4/25 [00:10<00:53,  2.56s/it]
Traceback (most recent call last):
  File "train_eval.py", line 23, in <module>
    cgnet.train(train)
  File "/content/ClimateNet/climatenet/models.py", line 113, in train
    self.optimizer.step()
  File "/usr/local/lib/python3.8/dist-packages/torch/optim/optimizer.py", line 113, in wrapper
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torch/autograd/grad_mode.py", line 27, in decorate_context
    return func(*args, **kwargs)
  File "/usr/local/lib/python3.8/dist-packages/torch/optim/adam.py", line 157, in step
    adam(params_with_grad,
  File "/usr/local/lib/python3.8/dist-packages/torch/optim/adam.py", line 213, in adam
    func(params,
  File "/usr/local/lib/python3.8/dist-packages/torch/optim/adam.py", line 305, in _single_tensor_adam
    denom = (exp_avg_sq.sqrt() / bias_correction2_sqrt).add_(eps)
KeyboardInterrupt
^C


# Save model for future evaluation

In [114]:
!ls models/TMQ-WS850-VRT850-PLS-.001-jl-mb16

config.json  weights.pth


In [129]:
!ls '/content/drive/MyDrive/Stanford/CS230/'

'Action Items.gdoc'	 'Final Report Office Hours.gdoc'
 ClimateNet		 'Links to all data.gsheet'
 Code			  Literature
 Data			 'Milestone Office Hours.gdoc'
 Experimentation.gsheet   Models


In [132]:
!mkdir /content/drive/MyDrive/Stanford/CS230/Models/TMQ-WS850-VRT850-PLS-.001-jl-mb16
!cp models/TMQ-WS850-VRT850-PLS-.001-jl-mb16/* /content/drive/MyDrive/Stanford/CS230/Models/TMQ-WS850-VRT850-PLS-.001-jl-mb16